# portfolio

In [131]:
import os
os.environ['NUMBA_DISABLE_JIT'] = '0'

# talib
uses candlestick patterns..
explanations of this voodoo magic bs are here: http://thepatternsite.com/CandleAlphabet.html

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import talib
import vectorbt as vbt

# Fetch price history
symbols = ['BTC-USD', 'ETH-USD', 'XRP-USD', 'BNB-USD', 'BCH-USD', 'LTC-USD']
start = '2020-01-01 UTC'  # crypto is UTC
end = '2020-09-01 UTC'
# OHLCV by column
ohlcv = vbt.YFData.download(symbols, start=start, end=end).concat()

The first that is nonzero is `CLD3INSIDE` info here: https://www.investopedia.com/terms/t/three-inside-updown.asp

In [6]:
# Run every single pattern recognition indicator and combine results
result = pd.DataFrame.vbt.empty_like(ohlcv['Open'], fill_value=0.)
for pattern in talib.get_function_groups()['Pattern Recognition']:
    PRecognizer = vbt.IndicatorFactory.from_talib(pattern)
    pr = PRecognizer.run(ohlcv['Open'], ohlcv['High'], ohlcv['Low'], ohlcv['Close'])
    if pr.integer.sum().sum() != 0:
        print(pattern)
        break
    result = result + pr.integer
    
# only use CLD3INSIDE for the patterns
result = pr.integer

CDL3INSIDE


Use that pattern as the full result

In [4]:
# Don't look into future
result = result.vbt.fshift(1)

# Treat each number as order value in USD
size = result / ohlcv['Open']

# Simulate portfolio
pf = vbt.Portfolio.from_orders(
    ohlcv['Close'], size, price=ohlcv['Open'],
    init_cash='autoalign', fees=0.001, slippage=0.001)

# Visualize portfolio value
pf.value().vbt.plot()

# my own

In [19]:
import re
import pandas as pd
idx = pd.IndexSlice
pd.options.display.max_columns = 100
from datetime import datetime

def parse_contract_symbol(cs):
    "expects contract symbol like 'WKHS200717C00002500' and returns formatted versions of gleenable info to add to df"
    if isinstance(cs,str):
        cs = cs
    else:
        if isinstance(cs['contract_symbol'],str):
            cs = cs['contract_symbol']
        else:
            return {
                "option_expiration": np.nan,
                "symbol": np.nan,
                "root": np.nan,
                "strike": np.nan,
                "type": np.nan,
            }            
        
    
    price_part = cs[-8:]
    option_type = cs[-9]
    date_part = cs[-15:-9]
    root_part = cs[:-15]
    symbol_part = re.match("[A-Z]*", root_part).group()

    price_part = int(price_part) / 1000
    type_part = {"C": "CALL", "P": "PUT"}[option_type]
    date_part = datetime.strptime(date_part, "%y%m%d").strftime("%Y-%m-%d")

    return {
        "option_expiration": date_part,
        "symbol": symbol_part,
        "root": root_part,
        "strike": price_part,
        "type": type_part,
    }

def convert_to_wide(df,oc_pred_details,oc_cols,s_cols,otm_exp_price=0.00001):
    cols = oc_cols + s_cols
    df_tmp = df.reset_index().set_index('oc_hx_date') #.set_index('oc_hx_symbol')
    df_tmp = df_tmp.sort_index()
    df_tmp = df_tmp.groupby(['oc_hx_date','oc_hx_contract_symbol'])[cols].max().unstack().dropna(how='all')
    df_tmp.index.name = 'Date'
    df_tmp.columns.names = ['oc_hx','oc_hx_contract_symbol']
    df_tmp.columns = df_tmp.columns.swaplevel(0,1)
    
    df_close = df_tmp.copy()
    df_close.loc[:,:] = 1.0
#     option_durations = df[['contract_symbol','option_expiration',"strike","type"]].drop_duplicates().dropna().to_dict('records')
#     for option_duration in option_durations:
    for option_duration in oc_pred_details:
        if option_duration["option_expiration"] in df_close.index:
            if option_duration["type"] == "CALL":
                if df_close.loc[option_duration['option_expiration'],idx[option_duration['contract_symbol'],"s_hx_close"]] > option_duration["strike"]:
                    continue
                else:
                    df_close.loc[option_duration['option_expiration'],idx[option_duration['contract_symbol'],oc_cols]] = 0.0
            elif option_duration["type"] == "PUT":
                if df_close.loc[option_duration['option_expiration'],idx[option_duration['contract_symbol'],"s_hx_close"]] < option_duration["strike"]:
                    continue
                else:
                    df_close.loc[option_duration['option_expiration'],idx[option_duration['contract_symbol'],oc_cols]] = 0.0

    df_tmp = (df_tmp*df_close).replace({0.0:otm_exp_price})
    
    multi_ind = list(zip(
            [parse_contract_symbol(c)['symbol'] for c in df_tmp.columns.get_level_values('oc_hx_contract_symbol')],
            [parse_contract_symbol(c)['option_expiration'] for c in df_tmp.columns.get_level_values('oc_hx_contract_symbol')],
            [parse_contract_symbol(c)['type'] for c in df_tmp.columns.get_level_values('oc_hx_contract_symbol')]
        ))
    df_tmp = df_tmp.vbt.stack_index(pd.MultiIndex.from_tuples(multi_ind, names=["symbol", "option_expiration", "option_type"]))
    return df_tmp

In [ ]:
df = pd.read_csv('/Users/xbno/Downloads/20210201_pred_bt_full_version_17_18_ex_sm.csv') # 50
df['oc_hx_date'] = pd.to_datetime(df['oc_hx_date'])
df['s_hx_date'] = pd.to_datetime(df['s_hx_date'])
df.set_index(["oc_hx_date", "oc_hx_contract_symbol"], inplace=True)
df = df.sort_index()
df.shape

df = df.rename({'oc_hx_hi_bff': 'oc_hx_hi',
 'oc_hx_lo_bff': 'oc_hx_lo',
 'oc_hx_open_bff': 'oc_hx_open',
 'oc_hx_close_bff': 'oc_hx_close',
 'oc_hx_volume_bff': 'oc_hx_volume'},axis=1)

df[['option_expiration','symbol','root','strike','type']] = df.apply(parse_contract_symbol,axis=1,result_type="expand")

oc_pred_details = df[['contract_symbol','option_expiration',"strike","type","y_pred","purch_date"]].drop_duplicates().dropna().to_dict('records')

In [112]:
df.head()

contract_symbol  y_pred purch_date  \
oc_hx_date oc_hx_contract_symbol                                      
2021-02-01 AHC210219C00002500                NaN     NaN        NaN   
           AHC210521C00002500                NaN     NaN        NaN   
           ARAY210219C00006000               NaN     NaN        NaN   
           ARAY210319P00005000               NaN     NaN        NaN   
           ARAY210618C00005000               NaN     NaN        NaN   

                                 purch_datetime  trade_price  \
oc_hx_date oc_hx_contract_symbol                               
2021-02-01 AHC210219C00002500               NaN          NaN   
           AHC210521C00002500               NaN          NaN   
           ARAY210219C00006000              NaN          NaN   
           ARAY210319P00005000              NaN          NaN   
           ARAY210618C00005000              NaN          NaN   

                                 option_expiration oc_hx_symbol  oc_hx_hi  \
oc_hx_date oc_hx_contract_symbol                                            
2021-02-01 AHC210219C00002500                  NaN          AHC      0.42   
           AHC210521C00002500                  NaN          AHC      0.65   
           ARAY210219C00006000                 NaN         ARAY      0.10   
           ARAY210319P00005000                 NaN         ARAY      0.50   
           ARAY210618C00005000                 NaN         ARAY      0.85   

                                  oc_hx_lo  oc_hx_open  oc_hx_close  \
oc_hx_date oc_hx_contract_symbol                                      
2021-02-01 AHC210219C00002500         0.30        0.40         0.30   
           AHC210521C00002500         0.60        0.60         0.65   
           ARAY210219C00006000        0.05        0.06         0.05   
           ARAY210319P00005000        0.50        0.50         0.50   
           ARAY210618C00005000        0.85        0.85         0.85   

                                  oc_hx_volume s_hx_symbol  s_hx_hi  s_hx_lo  \
oc_hx_date oc_hx_contract_symbol                                               
2021-02-01 AHC210219C00002500               18         AHC     8.76   8.4800   
           AHC210521C00002500                3         AHC     8.76   8.4800   
           ARAY210219C00006000              16        ARAY     5.05   4.8601   
           ARAY210319P00005000               5        ARAY     5.05   4.8601   
           ARAY210618C00005000               3        ARAY     5.05   4.8601   

                                  s_hx_open  s_hx_close  s_hx_volume  \
oc_hx_date oc_hx_contract_symbol                                       
2021-02-01 AHC210219C00002500          8.60        8.68      21416.0   
           AHC210521C00002500          8.60        8.68      21416.0   
           ARAY210219C00006000         5.02        4.93     932985.0   
           ARAY210319P00005000         5.02        4.93     932985.0   
           ARAY210618C00005000         5.02        4.93     932985.0   

                                  s_hx_date symbol root  strike type  
oc_hx_date oc_hx_contract_symbol                                      
2021-02-01 AHC210219C00002500    2021-02-01    NaN  NaN     NaN  NaN  
           AHC210521C00002500    2021-02-01    NaN  NaN     NaN  NaN  
           ARAY210219C00006000   2021-02-01    NaN  NaN     NaN  NaN  
           ARAY210319P00005000   2021-02-01    NaN  NaN     NaN  NaN  
           ARAY210618C00005000   2021-02-01    NaN  NaN     NaN  NaN

In [121]:
df_tmp = convert_to_wide(df,oc_pred_details,oc_cols=['trade_price','y_pred','oc_hx_open','oc_hx_hi','oc_hx_lo','oc_hx_close'],s_cols=["s_hx_close"])
# df_tmp = convert_to_wide(df,oc_pred_details,oc_cols=['oc_hx_close',"y_pred"],s_cols=["s_hx_close"])
# df_pred = df_tmp.loc[:,idx[:,:,:,:,"y_pred"]].copy()
# (df_tmp.loc[:,idx[:,:,:,:,"oc_hx_close"]].isnull() == False).sum().sort_values()


In [104]:
# bt = df.drop_duplicates(subset=["contract_symbol","purch_date","purch_datetime"]).dropna()

In [122]:
df_sm = df_tmp.loc[:,idx["CNET",:,:,:,:]].copy()

In [123]:
df_sm

symbol                               CNET                      \
option_expiration              2021-02-19                       
option_type                          CALL                       
oc_hx_contract_symbol CNET210219C00003000 CNET210219C00004000   
oc_hx                         trade_price         trade_price   
Date                                                            
2021-02-01                           0.34                 NaN   
2021-02-02                           0.30                 NaN   
2021-02-03                            NaN                0.12   
2021-02-04                            NaN                 NaN   
2021-02-05                           0.36                 NaN   
...                                   ...                 ...   
2021-06-23                            NaN                 NaN   
2021-06-24                            NaN                 NaN   
2021-06-25                            NaN                 NaN   
2021-06-28                            NaN                 NaN   
2021-06-29                            NaN                 NaN   

symbol                                                         \
option_expiration                                               
option_type                                                     
oc_hx_contract_symbol CNET210219C00005000 CNET210219C00006000   
oc_hx                         trade_price         trade_price   
Date                                                            
2021-02-01                            NaN                 NaN   
2021-02-02                            NaN                 NaN   
2021-02-03                            NaN                 NaN   
2021-02-04                            NaN                 NaN   
2021-02-05                            NaN                 NaN   
...                                   ...                 ...   
2021-06-23                            NaN                 NaN   
2021-06-24                            NaN                 NaN   
2021-06-25                            NaN                 NaN   
2021-06-28                            NaN                 NaN   
2021-06-29                            NaN                 NaN   

symbol                                                         \
option_expiration                                               
option_type                                                     
oc_hx_contract_symbol CNET210219C00007000 CNET210219C00008000   
oc_hx                         trade_price         trade_price   
Date                                                            
2021-02-01                            NaN                 NaN   
2021-02-02                            NaN                 NaN   
2021-02-03                            NaN                 NaN   
2021-02-04                            NaN                 NaN   
2021-02-05                            NaN                 NaN   
...                                   ...                 ...   
2021-06-23                            NaN                 NaN   
2021-06-24                            NaN                 NaN   
2021-06-25                            NaN                 NaN   
2021-06-28                            NaN                 NaN   
2021-06-29                            NaN                 NaN   

symbol                                                         \
option_expiration                                               
option_type                                               PUT   
oc_hx_contract_symbol CNET210219C00009000 CNET210219P00002000   
oc_hx                         trade_price         trade_price   
Date                                                            
2021-02-01                            NaN                 NaN   
2021-02-02                            NaN                 NaN   
2021-02-03                            NaN                 NaN   
2021-02-04                            NaN                 NaN   
2021-02-05                      

In [127]:
df_sm.loc[:,idx[:,:,:,"CNET210219C00003000",:]]

symbol                               CNET                                \
option_expiration              2021-02-19                                 
option_type                          CALL                                 
oc_hx_contract_symbol CNET210219C00003000                                 
oc_hx                         trade_price    y_pred oc_hx_open oc_hx_hi   
Date                                                                      
2021-02-01                           0.34  1.191465        0.3     0.35   
2021-02-02                           0.30  1.193882        0.3     0.30   
2021-02-03                            NaN       NaN        0.3     0.31   
2021-02-04                            NaN       NaN        0.3     0.30   
2021-02-05                           0.36  1.100981        0.3     0.40   
...                                   ...       ...        ...      ...   
2021-06-23                            NaN       NaN        NaN      NaN   
2021-06-24                            NaN       NaN        NaN      NaN   
2021-06-25                            NaN       NaN        NaN      NaN   
2021-06-28                            NaN       NaN        NaN      NaN   
2021-06-29                            NaN       NaN        NaN      NaN   

symbol                                                 
option_expiration                                      
option_type                                            
oc_hx_contract_symbol                                  
oc_hx                 oc_hx_lo oc_hx_close s_hx_close  
Date                                                   
2021-02-01                0.20        0.25       2.45  
2021-02-02                0.19        0.20       2.45  
2021-02-03                0.20        0.30       2.58  
2021-02-04                0.25        0.30       2.53  
2021-02-05                0.28        0.30       2.66  
...                        ...         ...        ...  
2021-06-23                 NaN         NaN        NaN  
2021-06-24                 NaN         NaN        NaN  
2021-06-25                 NaN         NaN        NaN  
2021-06-28                 NaN         NaN        NaN  
2021-06-29                 NaN         NaN        NaN  

[104 rows x 7 columns]

In [172]:
uniform_y_pred = vbt.IndicatorFactory(
    input_names=['y_pred', 'trade_price','oc_hx_lo'], 
#     input_names=['oc'], 
    param_names=['thresh','window'], 
    output_names=['entry']
)

In [179]:
@njit
def uniform_y_pred_entry(y_pred,trade_price,oc_hx_lo,thresh,window):
    return vbt.nb.rolling_max_nb(y_pred,window) > thresh    
    
# uniform_y_pred.from_apply_func(uniform_y_pred_entry).run(df_sm.loc[:,idx[:,:,:,"CNET210219C00003000",:]],1).entry
uniform_y_pred.from_apply_func(uniform_y_pred_entry).run(
    y_pred=df_sm.loc[:,idx[:,:,:,"CNET210219C00003000","y_pred"]].iloc[:,0],
    trade_price=df_sm.loc[:,idx[:,:,:,"CNET210219C00003000","trade_price"]].iloc[:,0],
    oc_hx_lo=df_sm.loc[:,idx[:,:,:,"CNET210219C00003000","oc_hx_lo"]].iloc[:,0],
    # works when doing: return y_pred > thresh
#     y_pred=df_sm.loc[:,idx[:,:,:,"CNET210219C00003000","y_pred"]],
#     trade_price=df_sm.loc[:,idx[:,:,:,"CNET210219C00003000","trade_price"]],
#     oc_hx_lo=df_sm.loc[:,idx[:,:,:,"CNET210219C00003000","oc_hx_lo"]],
    thresh=[1,1.15],
    window=[1,2]
).entry.head(25)

custom_thresh,1.00,1.15
custom_window,1,2
Date,,
2021-02-01,True,False
2021-02-02,True,True
2021-02-03,False,False
2021-02-04,False,False
2021-02-05,True,False
2021-02-08,True,True
2021-02-09,False,False
2021-02-10,False,False


In [106]:
uniform_y_pred = vbt.IndicatorFactory(input_names=['open', 'high'], param_names=['x'], output_names=['out'])\
    .from_apply_func(njit(lambda open, high, x: (open * 2 - high) < x))\
    .run(pd.Series([1, 2, 3]), pd.Series([1, 2, 3]), [1, 2]).out

custom_x,1,2
0,False,True
1,False,False
2,False,False


In [165]:
vbt.nb.rolling_mean_nb

CPUDispatcher(<function rolling_mean_nb at 0x11e621700>)

CPUDispatcher(<function rolling_max_nb at 0x11e621280>)

In [57]:
df_tmp.loc[:,idx["AHC",:,:,"AHC1210618C00005000",['oc_hx_hi']]].dropna().iloc[:,0]

Date
2021-06-15    0.20000
2021-06-16    0.05000
2021-06-17    0.05000
2021-06-18    0.00001
Name: (AHC, 2021-06-18, CALL, AHC1210618C00005000, oc_hx_hi), dtype: float64

In [69]:
vbt.STOCH.run(
df_tmp.loc[:,idx[:,:,:,"CNET210820C00005000",['oc_hx_hi']]].iloc[:,0],
df_tmp.loc[:,idx[:,:,:,"CNET210820C00005000",['oc_hx_lo']]].iloc[:,0],
df_tmp.loc[:,idx[:,:,:,"CNET210820C00005000",['oc_hx_close']]].iloc[:,0]
).plot()

FigureWidget({
    'data': [{'name': '%K',
              'showlegend': True,
              'type': 'scatter',
…

In [ ]:
df_tmp.loc[:,idx["AHC",:,:,"AHC1210618C00005000",['oc_hx_hi']]].iloc[:,0],
df_tmp.loc[:,idx["AHC",:,:,"AHC1210618C00005000",['oc_hx_lo']]].iloc[:,0],
df_tmp.loc[:,idx["AHC",:,:,"AHC1210618C00005000",['oc_hx_close']]].iloc[:,0]

In [52]:
df_tmp.loc[:,idx["AHC",:,:,"AHC1210618C00005000",['oc_hx_close']]].iloc[:,0]

Date
2021-02-01   NaN
2021-02-02   NaN
2021-02-03   NaN
2021-02-04   NaN
2021-02-05   NaN
              ..
2021-06-23   NaN
2021-06-24   NaN
2021-06-25   NaN
2021-06-28   NaN
2021-06-29   NaN
Name: (AHC, 2021-06-18, CALL, AHC1210618C00005000, oc_hx_close), Length: 104, dtype: float64

In [77]:
vbt.RSI.run(
    close=df_tmp.loc[:,idx[:,:,:,"CNET210820C00005000",['oc_hx_close']]].iloc[:,0],
    window=21
).plot()

FigureWidget({
    'data': [{'name': 'RSI',
              'showlegend': True,
              'type': 'scatter',…

In [90]:
bb = vbt.BBANDS.run(
    close=df_tmp.loc[:,idx[:,:,:,"CNET210820C00005000",['oc_hx_close']]].iloc[:,0],
)
bb.plot()

FigureWidget({
    'data': [{'line': {'color': 'rgba(127,127,127,0.7500)'},
              'name': 'Lower Band'…

In [98]:
ma = vbt.MA.run(df_tmp.loc[:,idx[:,:,:,"CNET210820C00005000",['oc_hx_close']]].iloc[:,0], 14)
ma.ma.shape

(104,)

# something else

In [11]:
import pandas as pd
from numba import njit
import vectorbt as vbt
from vectorbt.portfolio.nb import order_nb
import numpy as np
from vectorbt.portfolio.nb import close_position_nb

from vectorbt.portfolio.nb import sort_call_seq_nb
from vectorbt.portfolio.enums import SizeType, Direction

@njit
def pre_group_func_nb(c):
    '''Define empty arrays for each group.'''
    order_value_out = np.empty(c.group_len, dtype=np.float_)
    return (order_value_out,)

@njit
def pre_segment_func_nb(c, order_value_out):
    '''Perform rebalancing at each segment.'''
    for col in range(c.from_col, c.to_col):
        # Here we use order price for group valuation
        c.last_val_price[col] = c.close[c.i, col]
    # Reorder call sequence such that selling orders come first and buying last
    size = 1 / c.group_len
    size_type = SizeType.TargetPercent
    direction = Direction.LongOnly  # long positions only
    sort_call_seq_nb(c, size, size_type, direction, order_value_out)
    return (size, size_type, direction)

@njit
def order_func_nb(c, size, size_type, direction, fees, fixed_fees, slippage):
    '''Place an order.'''
    return order_nb(
        size=size,
        size_type=size_type,
        direction=direction,
        fees=fees,
        fixed_fees=fixed_fees,
        slippage=slippage
    )

np.random.seed(42)
close = np.random.uniform(1, 10, size=(5, 3))
fees = 0.001
fixed_fees = 1.
slippage = 0.001

pf = vbt.Portfolio.from_order_func(
    close,  # acts both as reference and order price here
    order_func_nb, fees, fixed_fees, slippage,  # order_args as *args
    segment_mask=1,  # rebalance every second tick
    pre_group_func_nb=pre_group_func_nb,
    pre_segment_func_nb=pre_segment_func_nb,
    cash_sharing=True, group_by=True,  # one group with cash sharing
    use_numba=False
)

pf.asset_value(group_by=False).vbt.plot()

FigureWidget({
    'data': [{'name': '0',
              'showlegend': True,
              'type': 'scatter',
 …

# simulate_nb

https://polakowo.io/vectorbt/docs/portfolio/nb.html#vectorbt.portfolio.nb.simulate_nb

In [9]:
import numpy as np
import pandas as pd
from numba import njit
from vectorbt.generic.plotting import Scatter
from vectorbt.records.nb import col_map_nb
from vectorbt.portfolio.enums import SizeType, Direction
from vectorbt.portfolio.nb import (
    order_nb,
    simulate_nb,
    simulate_row_wise_nb,
    build_call_seq,
    sort_call_seq_nb,
    asset_flow_nb,
    assets_nb,
    asset_value_nb,
)


@njit
def pre_sim_func_nb(c):
    print("before simulation")
    # Prepare empty arrays and pass them down the stack
    order_value_out = np.empty(c.target_shape[1], dtype=np.float_)
    return (order_value_out,)


@njit
def pre_group_func_nb(c, order_value_out):
    print("\tbefore group", c.group)
    # Forward down the stack
    return (order_value_out,)


@njit
def pre_segment_func_nb(c, order_value_out):
    print("\t\tbefore segment", c.i)
    # Reorder call sequence such that selling orders come first and buying last
    for col in range(c.from_col, c.to_col):
        # Here we use order price for group valuation (just for illustration!)
        c.last_val_price[col] = close[c.i, col]
    size = 1 / c.group_len
    size_type = SizeType.TargetPercent
    direction = Direction.LongOnly
    order_value_out = order_value_out[c.from_col : c.to_col]
    sort_call_seq_nb(c, size, size_type, direction, order_value_out)
    return (size, size_type, direction)


@njit
def order_func_nb(c, size, size_type, direction, fees, fixed_fees, slippage):
    print("\t\t\tcreating order", c.call_idx, "at column", c.col)
    # Create an order
    return order_nb(
        size=size,
        price=close[c.i, c.col],
        size_type=size_type,
        direction=direction,
        fees=fees,
        fixed_fees=fixed_fees,
        slippage=slippage,
    )


@njit
def post_order_func_nb(c, size, size_type, direction):
    print("\t\t\t\torder status:", c.order_result.status)


@njit
def post_segment_func_nb(c, order_value_out):
    print("\t\tafter segment", c.i)
    return None


@njit
def post_group_func_nb(c, order_value_out):
    print("\tafter group", c.group)
    return None


@njit
def post_sim_func_nb(c):
    print("after simulation")
    return None

In [10]:
vbt.IndicatorFactory(input_names=['open', 'high'], param_names=['x'], output_names=['out'])\
    .from_apply_func(njit(lambda open, high, x: (open * 2 - high) < x))\
    .run(pd.Series([1, 2, 3]), pd.Series([1, 2, 3]), [1, 2]).out

custom_x,1,2
0,False,True
1,False,False
2,False,False


In [ ]:
target_shape = (5, 3)
np.random.seed(42)
close = np.random.uniform(1, 10, size=target_shape)
group_lens = np.array([3])  # one group of three columns
init_cash = np.array([100.0])  # one capital per group
cash_sharing = True
call_seq = build_call_seq(target_shape, group_lens)  # will be overridden
segment_mask = np.array([True, False, True, False, True])[:, None]
segment_mask = np.copy(np.broadcast_to(segment_mask, target_shape))
fees = 0.001
fixed_fees = 1.0
slippage = 0.001

order_records, log_records = simulate_nb(
    target_shape=target_shape,
    close=close,
    group_lens=group_lens,
    init_cash=init_cash,
    cash_sharing=cash_sharing,
    call_seq=call_seq,
    segment_mask=segment_mask,
    pre_sim_func_nb=pre_sim_func_nb,
    pre_sim_args=(),
    post_sim_func_nb=post_sim_func_nb,
    post_sim_args=(),
    pre_group_func_nb=pre_group_func_nb,
    pre_group_args=(),
    post_group_func_nb=post_group_func_nb,
    post_group_args=(),
    pre_segment_func_nb=pre_segment_func_nb,
    pre_segment_args=(),
    post_segment_func_nb=post_segment_func_nb,
    post_segment_args=(),
    order_func_nb=order_func_nb,
    order_args=(fees, fixed_fees, slippage),
    post_order_func_nb=post_order_func_nb,
    post_order_args=(),
)